### 내부 데이터 전처리

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
from matplotlib.colors import rgb2hex
from matplotlib.colors import ListedColormap
import matplotlib
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina' #화질 좋게 해주기")
import plotly.express as px
import plotly.offline as pyo
import plotly.graph_objs as go
import sklearn
import seaborn as sb
import plotly.io as pio
import warnings
warnings.filterwarnings(action='ignore') 
import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy.stats import norm
import scipy.stats as stats

In [ ]:
dat = '/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/내부 데이터/'

# 데이터 읽는 함수 만들기

buycol = ['date', 'sex', 'age', 'bigcat', 'smcat', 'qty']
snscol = ['date', 'bigcat', 'smcat', 'cnt']

def read_buy(df):
    df = pd.read_csv(df +".csv")
    df = df.drop('Unnamed: 0', axis=1)
    df.columns = buycol
    return df
    
def read_sns(df):
    df = pd.read_csv(df +".csv")
    df = df.drop('Unnamed: 0', axis=1)
    df.columns = snscol
    return df

In [ ]:
buy1 = read_buy(dat+"buy2018_1")
buy2 = read_buy(dat+"buy2018_2")
buy3 = read_buy(dat+"buy2019_1")
buy4 = read_buy(dat+"buy2019_2")
buy = pd.concat([buy1, buy2, buy3, buy4])

In [ ]:
sns1 = read_sns(dat+"sns2018_1")
sns2 = read_sns(dat+"sns2018_2")
sns3 = read_sns(dat+"sns2019_1")
sns4 = read_sns(dat+"sns2019_2")
sns = pd.concat([sns1, sns2, sns3, sns4])

In [ ]:
# datetime으로 바꾸기
from datetime import datetime
buy['date'] = pd.to_datetime(buy['date'], format='%Y%m%d')
sns['date'] = pd.to_datetime(sns['date'], format='%Y%m%d')

In [ ]:
# 각 카테고리 별 총합
fig, ax = plt.subplots(1,2,figsize=(20,5))

# 주별로 묶기 전
df = buy.groupby(['date','bigcat']).sum().reset_index()
for i in df['bigcat'].unique().tolist():
    d_ = df[(df["bigcat"]==i)]
    ax[0].plot(d_["date"], d_["qty"], "-", alpha=.6)
ax[0].set_title('daily')

# 주별로 묶은 뒤
df = df.set_index('date')
plt.figure(figsize=(10,5))
for i in df['bigcat'].unique().tolist():
    d_ = df[(df["bigcat"]==i)]
    d_ = d_.resample('W', label='left').mean().reset_index() # 주 단위로 묶어주기
    ax[1].plot(d_["date"], d_["qty"], "-", alpha=.6)
ax[1].set_title('weekly')

plt.tight_layout()

### 외부 데이터 전처리

In [ ]:
# 사전에 전처리한 데이터들 합차기
df1 = pd.read_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/날씨 데이터/기온강수.csv')
df2 = pd.read_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/날씨 데이터/기온습도열지수_서울.csv')
df3 = pd.read_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/날씨 데이터/대기오염_서울.csv')

In [ ]:
# datetime으로 바꾸기
from datetime import datetime
df2['일자'] = pd.to_datetime(df2['일자'], format='%Y%m%d')

# merge를 위해 str로 바꾸기
df1['일자'] = df1['일시'].astype(str)
df2['일자'] = df2['일자'].astype(str)
df3['일자'] = df3['측정일자'].astype(str)

# 하나의 데이터로 합치기
from functools import reduce
dfs = [df1,df2,df3]
df_merge = reduce(lambda left, right: pd.merge(left, right, on='일자', how='outer'), dfs)

In [ ]:
# 필요없는 컬럼 삭제
df_merge = df_merge.drop(['Unnamed: 0_x','Unnamed: 0_y','측정일자', '일자','기온(°C)_y'], axis=1)

# 컬럼명 변경
df_merge = df_merge.rename({'일시':'일자', '기온(°C)_x':'기온(°C)'}, axis='columns')

# datetime으로 바꾸기
df_merge['일자'] = pd.to_datetime(df_merge['일자'], format='%Y-%m-%d')
df_merge.to_csv('climate.csv', index=False)

In [ ]:
# 날씨자료 수정(대기오염)
data_2018 = pd.read_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/원본 데이터/날씨 데이터/대기오염/일평균_대기환경_정보_2018년.csv')
data_2019 = pd.read_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/원본 데이터/날씨 데이터/대기오염/일평균_대기환경_정보_2019년.csv')

# 자료 합치기
data_air = pd.concat([data_2018, data_2019], axis=0)
data_air = data_air.loc[:,data_air.columns[2:]]
data_air.columns = ['미세먼지(㎍/㎥)', '초미세먼지(㎍/㎥)', '오존(ppm)', '이산화질소농도(ppm)', '일산화탄소농도(ppm)', '아황산가스농도(ppm)']
data_air.reset_index(drop=True,inplace=True)

In [ ]:
# 기존 climate 불러오기
df_climate = pd.read_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/날씨 데이터/climate.csv')

# 기존 climate 분할
df_climate_1 = df_climate.loc[:,df_climate.columns[1:9]]
df_climate_2 = df_climate.loc[:,df_climate.columns[15:17]]

# 수정된 자료 넣고 합치기
data_climate = [df_climate_1, data_air, df_climate_2]
data_climate = reduce(lambda  left,right: pd.concat([left,right],
                                            axis=1), data_climate)
data_climate

#os.chdir('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/날씨 데이터')
#data_climate.to_csv('climate2.csv')

In [ ]:
# 자료 추가 (풍속, 구름 데이터)
data_2018 = pd.read_csv("/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/원본 데이터/날씨 데이터/2018풍속눈구름.csv", encoding='cp949')
data_2019 = pd.read_csv("/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/원본 데이터/날씨 데이터/2019풍속눈구름.csv", encoding='cp949')

data_all = pd.concat([data_2018, data_2019], ignore_index=True)
data_all = data_all.drop(['적설(cm)', '지점', '지점명'], axis=1)

from datetime import datetime
data_all['일시'] = pd.to_datetime(data_all['일시'])
data_all['일시'] = data_all['일시'].dt.floor('D')

# 결측치는 0으로 간주
data_all = data_all.fillna(0)

# 시간 별 풍속, 전운량을 평균내어 일별 데이터로 생성
newdata = data_all.groupby('일시').mean().reset_index()

In [ ]:
# 최종데이터 climate3
climate3 = pd.concat([data_climate, newdata], axis = 1)
climate3 = climate3.drop(['일시'], axis=1)
climate3
# climate3.to_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/날씨 데이터/climate3.csv', index=False)

### 모델링을 위한 데이터 전처리

In [ ]:
# drink만 따로 추출
buy_drink = buy[buy['smcat'].str.contains('차|아이스티|밀크티|음료|커피|코코아|즙|식혜|요거트|탄산수|우유|주스|밀크|음용식초|두유|생수')].reset_index()
buy_drink = buy_drink.drop(['index','bigcat'], axis=1)

sns_drink = sns[sns['smcat'].str.contains('차|아이스티|밀크티|음료|커피|코코아|즙|식혜|요거트|탄산수|우유|주스|밀크|음용식초|두유|생수')].reset_index()
sns_drink = sns_drink.drop(['index','bigcat'], axis=1)

In [ ]:
# 날씨 데이터 불러오기
climate = pd.read_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/날씨 데이터/climate3.csv')

In [ ]:
# merge하기 위해 따로 string으로 바꿔주기
buy_drink['date'] = buy_drink['date'].astype(str)
sns_drink['date'] = sns_drink['date'].astype(str)
climate['일자'] = climate['일자'].astype(str)

In [ ]:
# 데이터 구축
all = pd.merge(buy_drink, sns_drink, on=['date','smcat'])
df = buy_drink.groupby(['date', 'sex', 'age']).count().reset_index()
df = df.drop(['smcat', 'qty'], axis=1)

for i in buy_drink['smcat'].unique().tolist():
  df1 = all[all['smcat']==i]
  df = pd.merge(df, df1, on=['date','sex','age'], how='outer')
  df = df.drop('smcat', axis=1)
  cntname = i + ' sns'
  df = df.rename({'qty': i, 'cnt': cntname}, axis='columns')

# 결측치 처리된 데이터 0으로 바꿔주기
model_drink_data = df.fillna(0)
model_drink_data.head()

In [ ]:
model_drink_data.to_csv('/content/gdrive/Shareddrives/날씨 빅데이터 콘테스트/외부 데이터/전처리 후 데이터/모델링용 데이터/model_drink_data.csv', index=False)